In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Border, Side
import xlwings as xw

In [2]:
def execute(f, par = []): #Добавляет проверки перед выполнением функции
    global errors
    errors.delete('1.0', 'end')
    try:
        try:
            f(*par) #запускаем функцию с переданными параметрами
        except PermissionError:
            raise Exception(f'Файл не может быть обновлён пока он открыт. Закройте файл "{graph_file_name}"')
        except ValueError as err:
            if str(err) == 'Value must be either numerical or a string containing a wildcard':
                raise Exception(f'Из-за применённых в файле фильтров, файл "{graph_file_name}" не может быть считан. \
Выставите все фильтры на (Выделить всё) и сохраните файл')
            else:
                raise ValueError(err)
    except Exception as e:
        errors.insert(1.0, str(e))

def open_task_file():
    global ppr, get_file_name, graph_file_name, graph_sheet_name, entry_machine, after_open_new_ttk_el, entry_machine_text
    #Получаем станок => формируем имя файла
    entry_machine_text = entry_machine.get()
    if (entry_machine_text):
        graph_file_name = get_file_name.loc[entry_machine.get(), 'Файл']
    else:
        raise Exception(f"Не найден файл {graph_file_name}")
    frame = pd.read_excel(graph_file_name, graph_sheet_name, skiprows=[0])
    ppr = frame
    # Переводим столбец 'Кто' в нижний регистр (игнорируем np.nan)
    ppr.loc[~ppr['Кто'].isna(), 'Кто'] = ppr.loc[~ppr['Кто'].isna(), 'Кто'].map(lambda x : x.lower())
    after_open_new_ttk_el()  # Добавляет новые элементы на интерфейс если файл != none


def format_sheet(worksheet):
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        max_length = max([len(str(cell.value)) for cell in col if cell.value != None])
        # перенос текста
        if (max_length > 100):
            worksheet.column_dimensions[column].width = 50
            for cell in col:
                cell.alignment = openpyxl.styles.Alignment(wrap_text=True)
        # автоподбор ширины столбца
        else:
            worksheet.column_dimensions[column].width = max_length+2
        # Устанавливаем жирные границы
        # for cell in col:
            # cell.border = Side(style='thick', color='00000000')
            # cell.border = Border(left = Side(border_style = 'double'), right = Side(border_style = 'double'), top = Side(border_style = 'double'), bottom = Side(border_style = 'double'))


def create_task_file():
    global graph_file_name, graph_sheet_name, ppr, who, week, entry_machine, entry_week
    # TEST
    global week_who_df
    
    # получаем week, who
    if (entry_week.get()):
        week = int(entry_week.get())
    else:
        raise Exception('Введите значение номера недели')
    if (entry_who.get()):
        who = entry_who.get()
    else:
        raise Exception('Введите значение плановой группы')
    

    # Формирование заданий
    task_cols_names = ['Узел', 'Название работы', 'Периодичность', 'Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии']
    # Добавляем новые столбцы в исходную таблицу
    new_cols = ['Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии']
    for col in new_cols:
        ppr[col] = np.nan
    # Формируем таблицы с еженедельным рабочим заданием
    # Прихотливым индексированием выбираем только те строчки где есть галочка в соответствующей неделе и где соответствующий столбец 'Кто'
    week_who_df = ppr.loc[  
        (ppr.loc[:, week] == '✓') & (ppr.loc[:, 'Кто'] == who),
        ['Узел', 'ПУНКТ ТО / НЕДЕЛЯ', 'ПЕРИОДИЧНОСТЬ', 'Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии', 'Кто']
    ]
    # Переименовываем колонки (столбец 'Кто' в итоговой таблице не нужен, он нужен только по группировке)
    week_who_df.columns = task_cols_names + ['Кто']
    # Группируем по узлам
    rz_file_name = f'РЗ_нед {week}_{who}_{entry_machine_text}.xlsx'
    with pd.ExcelWriter(rz_file_name) as writer: # По умолчанию используется движок openpyxl
        for node in week_who_df['Узел'].dropna().unique():
            week_who_node_df = week_who_df[week_who_df['Узел'] == node]
            info_df = pd.DataFrame([
                ["Наименование оборудования", node],
                ["N недели", week],
                ["Плановая группа", who],
                ["Подтверждение ответственного", np.nan],
                ["Подтверждение руководителя", np.nan]
            ])
            info_df.to_excel(writer, sheet_name=f'нед_{week}_{who}_{node}', header=False, index=False)
            week_who_node_df = week_who_node_df.reset_index()
            week_who_node_df = week_who_node_df[task_cols_names] #Убираем лишние столбцы('Кто')
            week_who_node_df.insert(0, 'N пп', week_who_node_df.index + 1)
            week_who_node_df.to_excel(writer, sheet_name=f'нед_{week}_{who}_{node}', startrow = len(info_df)+1, index=False)  # записываем без колонки кто
            #Форматируем лист по ширине
            worksheet = writer.sheets[f'нед_{week}_{who}_{node}']
            # format_sheet(worksheet)
    # Открываем файл
    # xw.Book(rz_file_name)

In [3]:
root = tk.Tk()
root.title('Формирование еженедельного задания')
root.geometry('750x500')
errors_label = ttk.Label(text='Ошибки:')
errors_label.place(x=50, y=210)
errors = tk.Text(root, width=80, height=10, foreground='red')
errors.place(x=50, y=240)

graph_sheet_name = 'ГОД'

get_file_name = pd.read_excel('Файл для ppr exe.xlsx', 'Файлы', index_col = 0)

# Добавляет новые элементы на интерфейс если файл успешно открылся
def after_open_new_ttk_el():
    global ppr, week_label, entry_week, who_label, entry_who, bt_generate_task
    
    if ppr is not None:
        file_label = ttk.Label(text=f'Считан файл {graph_file_name}')
        file_label.place(x=50, y=85)
        week_label = ttk.Label(text='Ввод номера недели')
        week_label.place(x=50, y=115)
        entry_week = ttk.Combobox(values=[i for i in range(1, 52+1)], state='readonly')
        entry_week.place(x=200, y=115)

        who_label = ttk.Label(text='Ввод плановой группы')
        who_label.place(x=50, y=145)
        entry_who = ttk.Combobox(values=list(ppr['Кто'].dropna().unique()), state='readonly')
        entry_who.place(x=200, y=145)

        bt_generate_task = tk.Button(root, text='Сформировать задание', width=30, height=1, command=lambda: execute(create_task_file))
        bt_generate_task.place(x=50, y=175)

ppr = None
machine_label = ttk.Label(text='Выбор станка')
machine_label.place(x=50, y=25)
entry_machine = ttk.Combobox(values=list(get_file_name.index), state='readonly', width=50)
entry_machine.place(x=200, y=25)
bt_generate_task = tk.Button(root, text='Считать файл', width=30, height=1, command=lambda: execute(open_task_file))
bt_generate_task.place(x=50, y=55)

root.mainloop()

In [6]:
entry_machine_text = "M232  ( RDC Goepfert 1624 )"
if (entry_machine_text):
    graph_file_name = get_file_name.loc[entry_machine_text, 'Файл']
else:
    raise Exception(f"Не найден файл {graph_file_name}")
frame = pd.read_excel(graph_file_name, graph_sheet_name, skiprows=[0])
ppr = frame
# Переводим столбец 'Кто' в нижний регистр (игнорируем np.nan)
ppr.loc[~ppr['Кто'].isna(), 'Кто'] = ppr.loc[~ppr['Кто'].isna(), 'Кто'].map(lambda x : x.lower())

ppr

,Unnamed: 0,ПЕРИОДИЧНОСТЬ,ПУНКТ ТО / НЕДЕЛЯ,Узел,Кто,"Маркировка, характеристики ЗИП",Номер по каталогу,мин,человек,"В год, ч/ч",...,43,44,45,46,47,48,49,50,51,52
0,NaN,Узел,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Устройство загрузки,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.203704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,ЕЖЕНЕДЕЛЬНО,"Убедиться в том, что детали, подвергающиеся ви...",Устройство загрузки,м,NaN,NaN,10.0,1.0,8.666667,...,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
3,1.0,NaN,Проверьте натяжение цепей и отладьте регулятор...,Устройство загрузки,м,NaN,NaN,15.0,1.0,13.000000,...,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
4,1.0,NaN,Проверять места соединений скрепками приводных...,Устройство загрузки,м,NaN,NaN,10.0,1.0,8.666667,...,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,18.0,РАЗ В 4 МЕСЯЦА,Произвести общую проверку люфта всех подшипник...,Верхний счетчик стапелей,м,NaN,NaN,30.0,1.0,1.444444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,18.0,NaN,"Проверить состояние бронзовых гаек, заменить н...",Верхний счетчик стапелей,м,NaN,NaN,10.0,1.0,0.481481,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,18.0,NaN,Проверить герметичность систем сжатого воздуха...,Верхний счетчик стапелей,м,NaN,NaN,20.0,1.0,0.962963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,18.0,NaN,Проверить состояние рельсов и ходовых роликов,Верхний счетчик стапелей,м,NaN,NaN,15.0,1.0,0.722222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
